### Import

In [ ]:
import sys

sys.path.append('../')
sys.path.append('../ride/')

from ride_pfa import graph_osm_loader
import ride_pfa.centroid_graph.centroids_graph_builder as cgb
import ride_pfa.path_finding as pfa 
import ride_pfa.clustering as cls
import ride_pfa.drawer as dw

### Graph download

In [ ]:
id = graph_osm_loader.osm_cities_example['Paris']
g = graph_osm_loader.get_graph(id)

### Extraction Path Finding

In [ ]:
cms_resolver = cls.LouvainCommunityResolver(resolution=1)

t, cg = cgb.CentroidGraphBuilder().build_with_time(g, cms_resolver, iterations=1)
t, len(cg.cms)

In [ ]:
baseline = pfa.Dijkstra(g)

In [ ]:
test_alg = pfa.ExtractionPfa(
    g = g,
    upper=pfa.Dijkstra(cg.g),
    down=pfa.Dijkstra(g),
    cluster='cluster'
)

In [ ]:
nodes = list(g.nodes())
s,t = nodes[0], nodes[1]

l,p = baseline.find_path(s,t)
extraction_l, extraction_p = test_alg.find_path(s,t)
l, extraction_l

In [ ]:
m = dw.draw_graph(g)
m = dw.draw_path(g, p, m=m)
m = dw.draw_path(g, extraction_p, m=m, color='green')

In [ ]:
m.show_in_browser()

### Heuristic Path Finding

In [ ]:
test_alg = pfa.MinClusterDistanceBuilder(
    workers=2
).build_astar(g, cms_resolver)

In [ ]:
l,p = baseline.find_path(s,t)
heuristic_l, heuristic_p = test_alg.find_path(s,t)
l, heuristic_l